## **¿Qué es una función asíncrona?**
Una función **asíncrona** es aquella que no se ejecuta de forma estrictamente lineal.

En lugar de bloquear todo el programa mientras espera que termine una operación lenta (como leer un archivo, consultar una base de datos o hacer una petición a internet), puede suspenderse temporalmente y ceder el control a otras tareas. Más adelante, cuando la operación pendiente termina, la función se reanuda justo donde se había quedado.

**Ejemplo en el día a día:**
- Síncrona: haces cola en el banco, esperas turno sin hacer nada.
- Asíncrona: sacas ticket en el banco, te vas a hacer otras cosas, y cuando llaman tu número vuelves y sigues.

![Descripción opcional](https://cdn.prod.website-files.com/5d2dd7e1b4a76d8b803ac1aa/5fc4d4d7dea01e43ee075dbf_B6oeReCTEE90L7UN3fOOyhgqQO_hY4OZvFUTmXcPMjYIByNrdkYaXRe0qaAxgz9BlenAkd4_hgcw5OXKk8mTLjn1u-BeqEoEtSbW-SPwLzCZ2yeW-3gPJLOoMbc910y6IvaVziHG.png)

### **¿Qué es `async def`?**
- Es una forma de definir funciones asíncronas en Python.
- No bloquean el hilo: cuando esperan (ej. red, disco, API), ceden el control al event loop para que otras tareas avancen.
- No crean nuevos hilos ni procesos → todo ocurre en un solo hilo.

### **Comparación**

| Aspecto          | Sync (`def`) ⚙️                        | Async (`async def`) ⚡                        |
| ---------------- | -------------------------------------- | -------------------------------------------- |
| Ejecución        | Secuencial, paso a paso                | Concurrente (tareas intercaladas en un loop) |
| Uso de CPU       | Un solo hilo, bloquea                  | Un solo hilo, no bloquea                     |
| Ideal para…      | Cálculo pesado (CPU)                   | Esperas de I/O (API, DB, red, disco)         |
| Escalabilidad    | Limitada (bloqueos)                    | Muy alta en I/O intensivo                    |
| Paralelismo real | ❌ (solo con multiprocessing o threads) | ❌ (no multiproceso, solo concurrente)        |


In [24]:
import asyncio
import time

# -------------------------
# Ejemplo: Simula una tarea lenta típica
# -------------------------

async def io_task_async(nombre):
    print(f'[ASYNC] Inicia {nombre}')
    await asyncio.sleep(2)
    print(f'[ASYNC] Termina {nombre}')

def io_task_sync(nombre):
    print(f'[SYNC ] Inicia {nombre}')
    time.sleep(2) 
    print(f'[SYNC ] Termina {nombre}')


# Ejecución secuencial (SYNC)
print('\n--- Ejecución SINCRÓNICA ---')
start = time.time()

# Las tareas se ejecutan una detrás de otra → total ≈ 4 segundos
io_task_sync('A')
io_task_sync('B')

print(f'\nTiempo total sync: {time.time() - start:.2f} segundos\n')

# Ejecución concurrente (ASYNC)
print('\n--- Ejecución ASÍNCRONA ---')
start = time.time()

# Ambas tareas se lanzan 'al mismo tiempo' dentro del event loop
# Mientras A espera su sleep, B puede avanzar.
await asyncio.gather(
    io_task_async('A'),
    io_task_async('B')
)

print(f'\nTiempo total async: {time.time() - start:.2f} segundos\n')



--- Ejecución SINCRÓNICA ---
[SYNC ] Inicia A
[SYNC ] Termina A
[SYNC ] Inicia B
[SYNC ] Termina B

Tiempo total sync: 4.02 segundos


--- Ejecución ASÍNCRONA ---
[ASYNC] Inicia A
[ASYNC] Inicia B
[ASYNC] Termina A
[ASYNC] Termina B

Tiempo total async: 2.01 segundos



In [23]:
# Ejemplo: tareas de cálculo intensivo (CPU-bound)

# Bloquenate
def cpu_task_sync(name):
    print(f'[CPU-SYNC] Inicia {name}')
    total = 0
    for i in range(10**7):
        total += i*i
    print(f'[CPU-SYNC] Termina {name} -> {total}')

# Bloqueante aunque sea async
async def cpu_task_async(name):
    print(f'[CPU-ASYNC] Inicia {name}')
    total = 0
    for i in range(10**7):
        total += i*i
    print(f'[CPU-ASYNC] Termina {name} -> {total}')


# --- Ejecución secuencial (CPU-bound) ---
print('\n--- Ejecución SÍNCRONA ---')
start = time.time()

cpu_task_sync('A')
cpu_task_sync('B')

print(f'\nTotal time sync: {time.time() - start:.2f} seconds\n')


# --- Ejecución "asíncrona" (CPU-bound) ---
print('\n--- Ejecución ASÍNCRONA ---')
start = time.time()

await asyncio.gather(
    cpu_task_async('A'),
    cpu_task_async('B')
)

print(f'\nTotal time async: {time.time() - start:.2f} seconds\n')


--- Ejecución SÍNCRONA ---
[CPU-SYNC] Inicia A
[CPU-SYNC] Termina A -> 333333283333335000000
[CPU-SYNC] Inicia B
[CPU-SYNC] Termina B -> 333333283333335000000

Total time sync: 1.09 seconds


--- Ejecución ASÍNCRONA ---
[CPU-ASYNC] Inicia A
[CPU-ASYNC] Termina A -> 333333283333335000000
[CPU-ASYNC] Inicia B
[CPU-ASYNC] Termina B -> 333333283333335000000

Total time async: 1.09 seconds



### **🚀 Casos de uso donde `async def` se desempeña mejor que `def`**
1. **Llamadas a APIs externas (I/O bound)**

    Cuando tu aplicación debe hacer muchas peticiones HTTP a servicios externos (APIs, web scraping, etc.), `async` permite que no se bloquee el programa mientras espera las respuestas.

2. **Procesamiento de muchos archivos pequeños (lectura/escritura)**

    Cuando lees o escribes miles de archivos (logs, CSVs, JSONs), lo más lento suele ser el disco (I/O). Con `async` puedes despachar muchas operaciones en paralelo sin bloquear.

3. **Notificaciones o mensajería en tiempo real**

    En una app que debe enviar correos, mensajes a Slack, notificaciones push, etc., `async` te permite enviar varios en paralelo sin que el servidor quede bloqueado.

4. **Streaming de datos (sensores, websockets, Kafka, etc.)**

    Cuando consumes datos en streaming, `async` permite leer, procesar y despachar eventos en paralelo sin bloquear la llegada de nuevos mensajes.

### **⚠️ Casos donde NO gana `async def`**

Procesamiento intensivo de CPU (ej: entrenar un modelo de ML, cálculos con NumPy, pandas pesado, etc.).
→ Ahí es mejor usar procesos en paralelo (`multiprocessing`, `joblib`, `Ray`, `Dask`), porque `async` no evita que el CPU se bloquee.

In [ ]:
import requests
import aiohttp

# Ejemplos: I/O con API lenta

def io_task_sync(name):
    print(f'[SYNC ] Start {name}')
    resp = requests.get('https://httpbin.org/delay/2')  # API con delay 2s
    print(f'[SYNC ] End {name} - Status: {resp.status_code}')


async def io_task_async(name):
    print(f'[ASYNC] Start {name}')
    async with aiohttp.ClientSession() as session:
        async with session.get('https://httpbin.org/delay/2') as resp:
            await resp.text()
            print(f'[ASYNC] End {name} - Status: {resp.status}')


# Ejecución secuencial (SYNC)
print('\n--- SYNC EXECUTION ---')
start = time.time()

io_task_sync('A')
io_task_sync('B')

print(f'\nTotal time sync: {time.time() - start:.2f} seconds\n')

# Ejecución concurrente (ASYNC)
print('\n--- ASYNC EXECUTION ---')
start = time.time()

await asyncio.gather(
    io_task_async('A'),
    io_task_async('B')
)

# # Tener en cuenta apra scripts .py
# asyncio.run(asyncio.gather(
#         io_task_async('A'),
#         io_task_async('B')
#     ))

print(f'\nTotal time async: {time.time() - start:.2f} seconds\n')



--- SYNC EXECUTION ---
[SYNC ] Start A
[SYNC ] End A - Status: 200
[SYNC ] Start B
[SYNC ] End B - Status: 200

Total time sync: 4.89 seconds


--- ASYNC EXECUTION ---
[ASYNC] Start A
[ASYNC] Start B
[ASYNC] End A - Status: 200
[ASYNC] End B - Status: 200

Total time async: 2.89 seconds

